##### 01. 클로링 실습 : requests 이용한 실습 

In [1]:
import requests
from bs4 import BeautifulSoup
import os

# 키워드
query = "사과"

url = f"https://www.google.com/search?q={query}&source=lnms&tbm=isch"

header = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}

response = requests.get(url, headers=header)

soup = BeautifulSoup(response.text, "html.parser")
img_tags = soup.find_all("img")

urls = []
for img in img_tags:
    try:
        urls.append(img["data-src"])
    except KeyError:
        try:
            urls.append(img["data-iurl"])
        except KeyError:
            try:
                urls.append(img["src"])
            except KeyError:
                pass

# 이미지 저장하기 위한 폴더 생성 
# if not os.path.exists(query):
#     os.makedirs(query)
os.makedirs(query, exist_ok=True)

for i, url in enumerate(urls):
    try:
        img_data = requests.get(url, headers=header).content
        with open(os.path.join(query, f"{query}_{i}.jpg"), "wb") as f:
            f.write(img_data)
    except:
        pass


##### 02. 클로링 실습 : requests 이용한 실습 (0바이트 데이터 제거)

In [17]:
import requests
from bs4 import BeautifulSoup
import os

query = "사과"
url = f"https://www.google.com/search?q={query}&source=lnms&tbm=isch"

header = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}

response = requests.get(url, headers=header)

soup = BeautifulSoup(response.text, "html.parser")
img_tags = soup.find_all("img")

urls = []
for img in img_tags:
    try:
        url = img["data-src"]
    except KeyError:
        url = img["src"]
    urls.append(url)

if not os.path.exists(query):
    os.makedirs(query)

for i, url in enumerate(urls):
    try:
        img_data = requests.get(url, headers=header).content
        file_path = os.path.join(query, f"{query}_{i}.jpg")
        with open(file_path, "wb") as f:
            f.write(img_data)
        if os.path.getsize(file_path) == 0:
            os.remove(file_path)
    except:
        pass

#### 03. 크롤링 실습 : Selenium 

In [14]:
!chmod +x chromedriver

In [92]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service

import time
import os 
import urllib, requests

##### 1. query, chromedriver 실행 
qurey ="사과"
service = Service("./chromedriver")
driver = webdriver.Chrome(service=service)

##### 2. query 검색창에 추가
driver.get("https://www.google.co.kr/imghp?h1=ko")
keyword = driver.find_element_by_xpath("/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input")
keyword.send_keys(query)

#### 3. query 검색창에 검색 실행 
driver.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/button').click()

### 4. 스크롤 자동으로 내리기 다가 더보기 버튼 나오면 클릭 하도록 하기 
print(f"{query} 스크롤 내리는 중...")
elem = driver.find_element_by_tag_name('body')
for i in range(60) :
    elem.send_keys(Keys.PAGE_DOWN)
    time.sleep(0.1)

try :
    driver.find_element_by_xpath('/html/body/div[2]/c-wiz/div[3]/div[1]/div/div/div/div[2]/div[1]/div[2]/div[2]/input').click()
    for i in range(60):
        elem.send_keys(Keys.PAGE_DOWN)
        time.sleep(0.1)
except:
    pass
    
#### 5. 이미지 개수 파악 하기 
links = []
images = driver.find_elements_by_css_selector("img.rg_i.Q4LuWd")
for image in images :
    
    if image.get_attribute('src') != None :
        links.append(image.get_attribute('src'))
    elif image.get_attribute('data-scr') != None :
        links.append(image.get_attribute('data-src'))
    elif image.get_attribute('data-iurl') != None :
        links.append(image.get_attribute('data-iurl'))
        

print(f"{query} 찾은 이미지 개수 : ", len(links))
time.sleep(1)


#### 6. 이미지 다운로드 
count = 0
for i in links : 
    start = time.time()
    url = i
    os.makedirs(f"./{query}_img_download/", exist_ok=True)
    while True:
        try:
            urllib.request.urlretrieve(url, f"./{query}_img_download/{str(count)}_{query}.png")
            print(f"{str(count + 1)} / {str(len(links))} / {query} / 다운로드 시간 : {str(time.time() - start)[:5]} 초")
            break
        except urllib.error.HTTPError as e:
            print(f"HTTPError 발생 ({e}): 재시도 중...")
            time.sleep(5)
        except Exception as e:
            print(f"Error 발생 ({e}): 재시도 중...")
            time.sleep(5)
        if time.time() - start > 60:
            print(f"{query} 이미지 다운로드 실패")
            break
    count = count + 1 
    
print(f"{query} 이미지 다운로드 완료")
driver.close()



/var/folders/0b/scwhsq4519352cxgdnxsz_180000gn/T/ipykernel_83728/1882523696.py:16: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  keyword = driver.find_element_by_xpath("/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input")
/var/folders/0b/scwhsq4519352cxgdnxsz_180000gn/T/ipykernel_83728/1882523696.py:20: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/button').click()


사과 스크롤 내리는 중...


/var/folders/0b/scwhsq4519352cxgdnxsz_180000gn/T/ipykernel_83728/1882523696.py:24: DeprecationWarning: find_element_by_tag_name is deprecated. Please use find_element(by=By.TAG_NAME, value=name) instead
  elem = driver.find_element_by_tag_name('body')
/var/folders/0b/scwhsq4519352cxgdnxsz_180000gn/T/ipykernel_83728/1882523696.py:30: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('/html/body/div[2]/c-wiz/div[3]/div[1]/div/div/div/div[2]/div[1]/div[2]/div[2]/input').click()
/var/folders/0b/scwhsq4519352cxgdnxsz_180000gn/T/ipykernel_83728/1882523696.py:39: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  images = driver.find_elements_by_css_selector("img.rg_i.Q4LuWd")


사과 찾은 이미지 개수 :  561
1 / 561 / 사과 / 다운로드 시간 : 0.002 초
2 / 561 / 사과 / 다운로드 시간 : 0.001 초
3 / 561 / 사과 / 다운로드 시간 : 0.000 초
4 / 561 / 사과 / 다운로드 시간 : 0.000 초
5 / 561 / 사과 / 다운로드 시간 : 0.000 초
6 / 561 / 사과 / 다운로드 시간 : 0.000 초
7 / 561 / 사과 / 다운로드 시간 : 0.000 초
8 / 561 / 사과 / 다운로드 시간 : 0.000 초
9 / 561 / 사과 / 다운로드 시간 : 0.001 초
10 / 561 / 사과 / 다운로드 시간 : 0.000 초
11 / 561 / 사과 / 다운로드 시간 : 0.000 초
12 / 561 / 사과 / 다운로드 시간 : 0.000 초
13 / 561 / 사과 / 다운로드 시간 : 0.000 초
14 / 561 / 사과 / 다운로드 시간 : 0.000 초
15 / 561 / 사과 / 다운로드 시간 : 0.000 초
16 / 561 / 사과 / 다운로드 시간 : 0.172 초
17 / 561 / 사과 / 다운로드 시간 : 0.156 초
18 / 561 / 사과 / 다운로드 시간 : 0.163 초
19 / 561 / 사과 / 다운로드 시간 : 0.154 초
20 / 561 / 사과 / 다운로드 시간 : 0.157 초
21 / 561 / 사과 / 다운로드 시간 : 0.140 초
22 / 561 / 사과 / 다운로드 시간 : 0.158 초
23 / 561 / 사과 / 다운로드 시간 : 0.142 초
24 / 561 / 사과 / 다운로드 시간 : 1.504 초
25 / 561 / 사과 / 다운로드 시간 : 0.470 초
26 / 561 / 사과 / 다운로드 시간 : 0.561 초
27 / 561 / 사과 / 다운로드 시간 : 0.291 초
28 / 561 / 사과 / 다운로드 시간 : 0.168 초
29 / 561 / 사과 / 다운로드 시간 : 0.191 초
30 

239 / 561 / 사과 / 다운로드 시간 : 0.156 초
240 / 561 / 사과 / 다운로드 시간 : 0.157 초
241 / 561 / 사과 / 다운로드 시간 : 0.188 초
242 / 561 / 사과 / 다운로드 시간 : 0.157 초
243 / 561 / 사과 / 다운로드 시간 : 0.159 초
244 / 561 / 사과 / 다운로드 시간 : 0.175 초
245 / 561 / 사과 / 다운로드 시간 : 1.163 초
246 / 561 / 사과 / 다운로드 시간 : 0.153 초
247 / 561 / 사과 / 다운로드 시간 : 0.153 초
248 / 561 / 사과 / 다운로드 시간 : 0.155 초
249 / 561 / 사과 / 다운로드 시간 : 0.154 초
250 / 561 / 사과 / 다운로드 시간 : 0.143 초
251 / 561 / 사과 / 다운로드 시간 : 0.144 초
252 / 561 / 사과 / 다운로드 시간 : 0.144 초
253 / 561 / 사과 / 다운로드 시간 : 0.165 초
254 / 561 / 사과 / 다운로드 시간 : 0.134 초
255 / 561 / 사과 / 다운로드 시간 : 0.159 초
256 / 561 / 사과 / 다운로드 시간 : 0.196 초
257 / 561 / 사과 / 다운로드 시간 : 0.155 초
258 / 561 / 사과 / 다운로드 시간 : 0.170 초
259 / 561 / 사과 / 다운로드 시간 : 0.143 초
260 / 561 / 사과 / 다운로드 시간 : 0.161 초
261 / 561 / 사과 / 다운로드 시간 : 0.177 초
262 / 561 / 사과 / 다운로드 시간 : 0.173 초
263 / 561 / 사과 / 다운로드 시간 : 0.183 초
264 / 561 / 사과 / 다운로드 시간 : 0.330 초
265 / 561 / 사과 / 다운로드 시간 : 0.162 초
266 / 561 / 사과 / 다운로드 시간 : 0.151 초
267 / 561 / 사과 / 다운로

475 / 561 / 사과 / 다운로드 시간 : 0.160 초
476 / 561 / 사과 / 다운로드 시간 : 0.155 초
477 / 561 / 사과 / 다운로드 시간 : 0.159 초
478 / 561 / 사과 / 다운로드 시간 : 0.138 초
479 / 561 / 사과 / 다운로드 시간 : 0.145 초
480 / 561 / 사과 / 다운로드 시간 : 0.165 초
481 / 561 / 사과 / 다운로드 시간 : 0.160 초
482 / 561 / 사과 / 다운로드 시간 : 0.355 초
483 / 561 / 사과 / 다운로드 시간 : 0.172 초
484 / 561 / 사과 / 다운로드 시간 : 0.145 초
485 / 561 / 사과 / 다운로드 시간 : 0.161 초
486 / 561 / 사과 / 다운로드 시간 : 0.168 초
487 / 561 / 사과 / 다운로드 시간 : 0.146 초
488 / 561 / 사과 / 다운로드 시간 : 0.148 초
489 / 561 / 사과 / 다운로드 시간 : 0.161 초
490 / 561 / 사과 / 다운로드 시간 : 0.148 초
491 / 561 / 사과 / 다운로드 시간 : 0.157 초
492 / 561 / 사과 / 다운로드 시간 : 0.185 초
493 / 561 / 사과 / 다운로드 시간 : 0.147 초
494 / 561 / 사과 / 다운로드 시간 : 0.158 초
495 / 561 / 사과 / 다운로드 시간 : 0.166 초
496 / 561 / 사과 / 다운로드 시간 : 0.169 초
497 / 561 / 사과 / 다운로드 시간 : 0.161 초
498 / 561 / 사과 / 다운로드 시간 : 0.164 초
499 / 561 / 사과 / 다운로드 시간 : 0.178 초
500 / 561 / 사과 / 다운로드 시간 : 1.164 초
501 / 561 / 사과 / 다운로드 시간 : 0.165 초
502 / 561 / 사과 / 다운로드 시간 : 0.166 초
503 / 561 / 사과 / 다운로